# Airbnb Hotel Booking Analysis

This notebook follows the VOIS project guidelines for **AIRBNB Hotel Booking Analysis**.  

**How to use:**
- Place your dataset file named `airbnb_hotel_booking.csv` in the same folder as this notebook. If you don't have the file, the notebook will run with a small synthetic sample so you can see the workflow.
- Run all cells from top to bottom.

---

**What this notebook contains:**
1. Data loading
2. Data preview
3. Cleaning & preprocessing
4. Exploratory Data Analysis (EDA)
5. Visualizations (matplotlib)
6. Insights and export



In [ ]:
# 1. Setup: imports and global settings
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

# Make plots appear inline (Jupyter)
%matplotlib inline

# Create output folder for saved images and cleaned data
OUT_DIR = '/mnt/data/vois_airbnb_outputs'
os.makedirs(OUT_DIR, exist_ok=True)

print('Output directory:', OUT_DIR)


In [ ]:
# 2. Load dataset (tries to read 'airbnb_hotel_booking.csv').
# If file not found, create a synthetic sample dataset so the notebook cells can run end-to-end.
DATA_PATH = 'airbnb_hotel_booking.csv'

if os.path.exists(DATA_PATH):
    df = pd.read_csv(DATA_PATH)
    print('Loaded dataset from', DATA_PATH)
else:
    print(f"File '{DATA_PATH}' not found. Creating a synthetic sample dataset to demonstrate the workflow.")
    rng = np.random.default_rng(42)
    n = 300
    df = pd.DataFrame({
        'id': np.arange(1, n+1),
        'city': rng.choice(['CityA', 'CityB', 'CityC'], size=n, p=[0.5,0.3,0.2]),
        'room_type': rng.choice(['Entire home/apt','Private room','Shared room'], size=n, p=[0.6,0.3,0.1]),
        'price': rng.integers(800, 8000, size=n),
        'minimum_nights': rng.integers(1, 14, size=n),
        'number_of_reviews': rng.integers(0, 200, size=n),
        'reviews_per_month': np.round(rng.uniform(0,5,size=n),2),
        'availability_365': rng.integers(0,365,size=n),
        'last_review': pd.to_datetime('2025-01-01') - pd.to_timedelta(rng.integers(0,365,size=n), unit='d'),
        'host_since': pd.to_datetime('2018-01-01') - pd.to_timedelta(rng.integers(0,2000,size=n), unit='d'),
        'neighbourhood_group': rng.choice(['A','B','C'], size=n)
    })
    # Save synthetic CSV so user can inspect it
    df.to_csv(os.path.join(OUT_DIR, 'synthetic_airbnb_sample.csv'), index=False)
    print('Saved synthetic sample to', os.path.join(OUT_DIR, 'synthetic_airbnb_sample.csv'))

# Show basic info
print('Dataset shape:', df.shape)
df.head()


## 3. Basic info & Data Types
Check for missing values and data types.

In [ ]:
df.info()
print('\nMissing values per column:')
print(df.isnull().sum())

## 4. Quick summary statistics for numerical columns

In [ ]:
df.describe().T

## 5. Cleaning steps (examples)
- Convert date columns to datetime
- Fill or drop obvious missing values
- Remove duplicates
- Create derived columns (year/month)

In [ ]:
# Example cleaning pipeline (works safely even if columns are missing)
df_clean = df.copy()

# Convert common date columns if present
for col in ['last_review','host_since','date']:
    if col in df_clean.columns:
        df_clean[col] = pd.to_datetime(df_clean[col], errors='coerce')

# Drop duplicates if 'id' present
if 'id' in df_clean.columns:
    df_clean = df_clean.drop_duplicates(subset='id')
else:
    df_clean = df_clean.drop_duplicates()

# Fill numeric NaNs with 0 for demonstration (user may want other strategies)
num_cols = df_clean.select_dtypes(include=[np.number]).columns.tolist()
df_clean[num_cols] = df_clean[num_cols].fillna(0)

# Fill categorical NaNs with 'Unknown'
cat_cols = df_clean.select_dtypes(include=['object']).columns.tolist()
df_clean[cat_cols] = df_clean[cat_cols].fillna('Unknown')

# Derived columns: host_age_days (if host_since exists), last_review_month
if 'host_since' in df_clean.columns:
    df_clean['host_age_days'] = (pd.to_datetime('today') - df_clean['host_since']).dt.days
if 'last_review' in df_clean.columns:
    df_clean['last_review_month'] = df_clean['last_review'].dt.to_period('M').astype(str)

print('Cleaned shape:', df_clean.shape)
df_clean.head()

## 6. Exploratory Data Analysis (EDA)
We'll look at distributions, top categories and correlations.

In [ ]:
# 6.1 Top 10 cities / neighbourhoods
cat_col = None
for c in ['city','neighbourhood','neighbourhood_group']:
    if c in df_clean.columns:
        cat_col = c
        break

if cat_col:
    top_cats = df_clean[cat_col].value_counts().head(10)
    print('Top categories in', cat_col)
    display(top_cats)
else:
    print('No city/neighbourhood-like column found.')

In [ ]:
# 6.2 Price distribution (histogram) - numeric check
if 'price' in df_clean.columns:
    plt.figure(figsize=(8,4))
    plt.hist(df_clean['price'].dropna(), bins=30)
    plt.title('Price distribution')
    plt.xlabel('Price')
    plt.ylabel('Count')
    plt.tight_layout()
    plt.savefig(os.path.join(OUT_DIR, 'price_histogram.png'))
    print('Saved: price_histogram.png ->', os.path.join(OUT_DIR, 'price_histogram.png'))
    plt.show()
else:
    print('No price column found.')

In [ ]:
# 6.3 Boxplot of price by room_type (if both columns exist)
if set(['price','room_type']).issubset(df_clean.columns):
    plt.figure(figsize=(8,5))
    # prepare data
    groups = df_clean.groupby('room_type')['price'].apply(list)
    # create boxplots
    plt.boxplot(groups, labels=groups.index, showfliers=False)
    plt.title('Price by Room Type (no outliers shown)')
    plt.xlabel('Room Type')
    plt.ylabel('Price')
    plt.tight_layout()
    plt.savefig(os.path.join(OUT_DIR, 'price_by_roomtype_boxplot.png'))
    print('Saved: price_by_roomtype_boxplot.png ->', os.path.join(OUT_DIR, 'price_by_roomtype_boxplot.png'))
    plt.show()
else:
    print('Required columns for boxplot not found.')

In [ ]:
# 6.4 Correlation heatmap for numerical columns
num = df_clean.select_dtypes(include=[np.number])
if num.shape[1] >= 2:
    corr = num.corr()
    plt.figure(figsize=(8,6))
    plt.imshow(corr, aspect='auto')
    plt.colorbar()
    plt.xticks(range(len(corr.columns)), corr.columns, rotation=90)
    plt.yticks(range(len(corr.index)), corr.index)
    plt.title('Correlation matrix (numerical features)')
    plt.tight_layout()
    plt.savefig(os.path.join(OUT_DIR, 'correlation_matrix.png'))
    print('Saved: correlation_matrix.png ->', os.path.join(OUT_DIR, 'correlation_matrix.png'))
    plt.show()
else:
    print('Not enough numerical columns for correlation matrix.')

## 7. Grouped Analysis & Insights
Examples: average price by city and room type, busiest months (if dates available).

In [ ]:
# 7.1 Average price by category (city or neighbourhood-like)
if cat_col and 'price' in df_clean.columns:
    avg_price = df_clean.groupby(cat_col)['price'].mean().sort_values(ascending=False).head(10)
    print('Average price by', cat_col)
    display(avg_price)
    # Save to CSV for PPT use
    avg_price.to_csv(os.path.join(OUT_DIR, 'avg_price_by_category.csv'))
    print('Saved: avg_price_by_category.csv ->', os.path.join(OUT_DIR, 'avg_price_by_category.csv'))
else:
    print('Cannot compute avg price by category - missing columns.')

In [ ]:
# 7.2 Reviews vs Price scatter (if available)
if set(['price','number_of_reviews']).issubset(df_clean.columns):
    plt.figure(figsize=(7,4))
    plt.scatter(df_clean['price'], df_clean['number_of_reviews'], alpha=0.6)
    plt.xlabel('Price')
    plt.ylabel('Number of reviews')
    plt.title('Reviews vs Price')
    plt.tight_layout()
    plt.savefig(os.path.join(OUT_DIR, 'reviews_vs_price_scatter.png'))
    print('Saved: reviews_vs_price_scatter.png ->', os.path.join(OUT_DIR, 'reviews_vs_price_scatter.png'))
    plt.show()
else:
    print('Required columns for reviews vs price not found.')

## 8. Time-based analysis (if date or last_review exists)
Look at bookings/reviews over time if possible.

In [ ]:
if 'last_review' in df_clean.columns:
    df_time = df_clean.dropna(subset=['last_review']).copy()
    df_time['review_date'] = pd.to_datetime(df_time['last_review'], errors='coerce')
    df_time = df_time.dropna(subset=['review_date'])
    df_time['review_month'] = df_time['review_date'].dt.to_period('M').astype(str)
    monthly_counts = df_time['review_month'].value_counts().sort_index()
    plt.figure(figsize=(10,4))
    plt.plot(monthly_counts.index, monthly_counts.values, marker='o')
    plt.xticks(rotation=45)
    plt.title('Monthly review counts (from last_review column)')
    plt.tight_layout()
    plt.savefig(os.path.join(OUT_DIR, 'monthly_review_counts.png'))
    print('Saved: monthly_review_counts.png ->', os.path.join(OUT_DIR, 'monthly_review_counts.png'))
    plt.show()
else:
    print('No last_review column for time-based analysis.')

## 9. Save cleaned dataset and sample results for PPT
We'll save some CSVs and the top images for inclusion in your PPT slides.

In [ ]:
# Export cleaned dataset and a small summary for PPT
cleaned_path = os.path.join(OUT_DIR, 'airbnb_cleaned.csv')
df_clean.to_csv(cleaned_path, index=False)
print('Saved cleaned dataset to', cleaned_path)

# Create a small summary file
summary = {
    'rows_original': int(df.shape[0]),
    'rows_cleaned': int(df_clean.shape[0]),
    'columns': list(df_clean.columns),
}
import json
with open(os.path.join(OUT_DIR, 'summary.json'), 'w') as f:
    json.dump(summary, f, indent=2)
print('Saved summary.json ->', os.path.join(OUT_DIR, 'summary.json'))

## 10. Example Insights (write these into your PPT slide)
- Top 3 neighbourhoods by number of listings
- Average price range and outlier behaviour
- Relationship between reviews and price
- Seasonal trends in reviews/bookings (if present)

> Paste at least 3 result screenshots from the `vois_airbnb_outputs` folder into your PPT.

In [ ]:
# Show files saved into output for easy reference
for f in sorted(os.listdir(OUT_DIR)):
    print('-', f)

----

### End of notebook

You can now run this notebook. If you want, I can also:
- Run the notebook with your uploaded CSV and provide the generated screenshots, or
- Create the PPT slides and insert the generated images and GitHub placeholder info.

Tell me which you'd like next!